🔧 **Setup Required**: Before running this notebook, please follow the [setup instructions](../README.md#setup-instructions) to configure your environment and API keys.

# Building a Tool-Calling AI Agent with Haystack

This tutorial shows you how to build an AI agent that can autonomously use external tools (like web search) to answer questions.

## What You'll Learn

- What tool-calling agents are and why they're useful
- How to wrap Haystack components as tools for LLMs
- How to build a decision loop for tool selection
- Best practices for agentic workflows

## What is a Tool-Calling Agent?

A tool-calling agent can:
1. Recognize when it needs external information
2. Request specific tools to get that information
3. Process the tool's output
4. Generate a final answer using the results

Think of it like a researcher who knows when to search the web or use a calculator.

## Pipeline Flow

```
User Question → LLM → Router
                       ├─→ Tool Needed? → Execute Tool → Back to LLM
                       └─→ No Tool? → Return Answer
```

### Key Components

1. **Tool Definition** - Wrap functionality (web search) as a tool
2. **Generator (LLM)** - Decides when to use tools
3. **Router** - Routes based on whether tool calls are present
4. **Tool Invoker** - Executes the requested tool
5. **Message Collector** - Maintains conversation history

Let's build it!

## Step 1: Import Components

We need these Haystack components:

- **Pipeline** - Container for connecting components
- **ToolInvoker** - Executes tools requested by the LLM
- **OpenAIChatGenerator** - LLM that decides when to use tools
- **ConditionalRouter** - Routes messages based on tool calls
- **SearchApiWebSearch** - Web search tool
- **ComponentTool** - Converts components into LLM-callable tools
- **ChatMessage** - Message structure

We'll also create a **MessageCollector** to manage conversation history.

In [5]:
from haystack import component, Pipeline
from haystack.components.tools import ToolInvoker
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.routers import ConditionalRouter
from haystack.components.websearch import SearchApiWebSearch
from haystack.core.component.types import Variadic
from haystack.dataclasses import ChatMessage
from haystack.tools import ComponentTool
from dotenv import load_dotenv
from haystack.utils import Secret
import os
from pathlib import Path

# Load .env from the root of ch8 directory
root_dir = Path(__file__).parent.parent if "__file__" in globals() else Path.cwd().parent
load_dotenv(root_dir / ".env")

from typing import Any, Dict, List

## Step 2: The MessageCollector Component

**Why we need it:** When the LLM requests a tool, we need to:
1. Remember the original question
2. Collect the tool's response
3. Send both back to the LLM for the final answer

**What it does:**
- Stores all messages (queries, tool calls, results)
- Combines message lists using `Variadic[List[ChatMessage]]`
- Returns complete conversation history to the LLM
- Can be cleared with `clear()` for fresh conversations

In [6]:
# helper component to temporarily store last user query before the tool call 
@component()
class MessageCollector:
    def __init__(self):
        self._messages = []

    @component.output_types(messages=List[ChatMessage])
    def run(self, messages: Variadic[List[ChatMessage]]) -> Dict[str, Any]:

        self._messages.extend([msg for inner in messages for msg in inner])
        return {"messages": self._messages}

    def clear(self):
        self._messages = []



## Step 3: Create a Web Search Tool

We wrap a web search component so the LLM can use it:

**ComponentTool** converts any Haystack component into a tool that:
- The LLM can "see" and understand
- The LLM can request by name when needed
- The ToolInvoker can execute automatically

**Configuration:**
- `top_k=5` - Returns top 5 search results
- `api_key` - Authenticates with SearchAPI
- `allowed_domains` - Optional: restrict to specific domains

The LLM will automatically call this tool when it needs current web information.

In [7]:
# Create a tool from a component
web_tool = ComponentTool(
    component=SearchApiWebSearch(top_k=5,
                                api_key=Secret.from_env_var("SEARCH_API_KEY"),
                                allowed_domains=["https://www.britannica.com/"])
)


## Step 4: Define Routing Logic

The **ConditionalRouter** checks if the LLM's response contains tool calls:

**Route 1: Tool Call Detected**
- Condition: `{{replies[0].tool_calls | length > 0}}`
- Action: Execute the tool via ToolInvoker
- Output: `there_are_tool_calls`

**Route 2: No Tool Call**
- Condition: `{{replies[0].tool_calls | length == 0}}`
- Action: Return the final answer
- Output: `final_replies`

**How It Works:**
1. LLM generates a response (with or without tool calls)
2. Router checks for tool calls
3. If tool calls exist → execute and loop back to LLM
4. If no tool calls → return the answer

In [8]:
# Define routing conditions
routes = [
    {
        "condition": "{{replies[0].tool_calls | length > 0}}",
        "output": "{{replies}}",
        "output_name": "there_are_tool_calls",
        "output_type": List[ChatMessage],
    },
    {
        "condition": "{{replies[0].tool_calls | length == 0}}",
        "output": "{{replies}}",
        "output_name": "final_replies",
        "output_type": List[ChatMessage], 
    },
]



## Step 5: Build and Connect the Pipeline

**Components:**
1. **message_collector** - Stores conversation history
2. **generator** - OpenAI LLM with tool access
3. **router** - Decides to invoke tools or return answer
4. **tool_invoker** - Executes tool calls

**Connection Flow:**

```
generator → router
    ├─→ tool_invoker (execute)
    ├─→ message_collector (store)
    ↓
tool results → message_collector → generator (feedback loop)
```

**The Feedback Loop:**
1. Generator creates tool call
2. Tool executes and stores results
3. Complete history goes back to Generator
4. Generator uses results for final answer

This cycle allows the agent to use tools iteratively until it can answer the question.

In [9]:
# Create the pipeline
tool_agent = Pipeline()
tool_agent.add_component("message_collector", MessageCollector())
tool_agent.add_component("generator", OpenAIChatGenerator(model="gpt-4o-mini", tools=[web_tool]))
tool_agent.add_component("router", ConditionalRouter(routes, unsafe=True))
tool_agent.add_component("tool_invoker", ToolInvoker(tools=[web_tool]))

tool_agent.connect("generator.replies", "router")
tool_agent.connect("router.there_are_tool_calls", "tool_invoker")
tool_agent.connect("router.there_are_tool_calls", "message_collector")
tool_agent.connect("tool_invoker.tool_messages", "message_collector")
tool_agent.connect("message_collector", "generator.messages")

2025-11-17T04:57:02.739211Z [warning  ] Unsafe mode is enabled. This allows execution of arbitrary code in the Jinja template. Use this only if you trust the source of the template. lineno=199 module=haystack.components.routers.conditional_router


🚅 Components
  - message_collector: MessageCollector
  - generator: OpenAIChatGenerator
  - router: ConditionalRouter
  - tool_invoker: ToolInvoker
🛤️ Connections
  - message_collector.messages -> generator.messages (List[ChatMessage])
  - generator.replies -> router.replies (list[ChatMessage])
  - router.there_are_tool_calls -> tool_invoker.messages (List[ChatMessage])
  - router.there_are_tool_calls -> message_collector.messages (List[ChatMessage])
  - tool_invoker.tool_messages -> message_collector.messages (list[ChatMessage])

## Step 6: Visualize the Pipeline

Let's draw the pipeline to see how components connect and data flows.

In [3]:
tool_agent.draw(path="./images/tool_agent_pipeline.png")

### Pipeline Diagram

![](./images/tool_agent_pipeline.png)

**Key Features:**

1. **Entry** - Messages enter through the generator (LLM)
2. **Decision** - Router checks for tool calls
3. **Tool Path** - If detected → tool_invoker executes → results to message_collector
4. **Feedback Loop** - message_collector sends history back to generator
5. **Exit** - No tool call → final answer via router.final_replies

**Notice:**
- Circular connection enables iterative tool use
- Router splits into two paths (tool vs. answer)
- message_collector accumulates the full conversation

## Step 7: Run the Agent

Let's test with a question requiring current information.

**Messages:**
1. System: "You're a helpful agent choosing the right tool when necessary"
2. User: "How is the weather in Berlin?"

**Expected Flow:**
1. LLM recognizes need for current data
2. Generates web search tool call
3. Router detects tool call → routes to tool_invoker
4. Web search executes and returns results
5. MessageCollector combines everything
6. LLM generates final answer using search results
7. Router returns the answer

Let's see it work!

In [4]:
messages = [
    ChatMessage.from_system("You're a helpful agent choosing the right tool when necessary"), 
    ChatMessage.from_user("How is the weather in Berlin?")]
result = tool_agent.run({"messages": messages})

print(result["router"]["final_replies"][0].text)

The search did not provide specific current weather information for Berlin. However, you can check reliable weather websites or apps for the most accurate and updated weather conditions. Would you like me to search again or provide guidance on where to look?


## Understanding the Output

The agent successfully:
1. ✅ Recognized it needed current information
2. ✅ Called the web search tool automatically
3. ✅ Retrieved search results
4. ✅ Created a natural language answer

**Behind the Scenes:**
```
User asks question
→ LLM: "I need current data, searching web"
→ Router detects tool call
→ Tool executes web search
→ Results collected
→ LLM: "Based on search, here's the weather..."
→ Router returns final answer
```

**Key Points:**
- **Autonomous** - Agent decided to use the tool
- **Iterative** - Looped through components
- **Context-Aware** - Combined search results with understanding
- **Transparent** - User just got an answer

In [ ]:
# ============================================================================
# Additional Examples and Experiments
# ============================================================================

# Example 1: Question that doesn't need a tool
print("="*80)
print("Example 1: Simple factual question")
print("="*80)

messages_simple = [
    ChatMessage.from_system("You're a helpful agent choosing the right tool when necessary"),
    ChatMessage.from_user("What is 25 + 37?")
]
result_simple = tool_agent.run({"messages": messages_simple})
print(f"Question: What is 25 + 37?")
print(f"Answer: {result_simple['router']['final_replies'][0].text}")
print()

# Example 2: Question that needs current information
print("="*80)
print("Example 2: Current events question")
print("="*80)

messages_current = [
    ChatMessage.from_system("You're a helpful agent choosing the right tool when necessary"),
    ChatMessage.from_user("What are the latest developments in AI technology?")
]
result_current = tool_agent.run({"messages": messages_current})
print(f"Question: What are the latest developments in AI technology?")
print(f"Answer: {result_current['router']['final_replies'][0].text}")
print()

# Note: Clear message collector between runs if needed for fresh context
# tool_agent.get_component("message_collector").clear()

## Key Takeaways

### What We Built

A **tool-calling agent** with:
1. Autonomous decision-making
2. Tool integration (web search)
3. Iterative processing
4. Conversation memory

### When to Use Tool-Calling Agents

**Good for:**
- Questions requiring current/external information
- Tasks needing calculations or specialized processing
- Multi-step reasoning with data retrieval

**Not ideal for:**
- Simple Q&A (LLM already knows)
- High-speed requirements (tool calls add latency)
- Fully deterministic workflows

### Try These Extensions

1. Add more tools (calculator, database, code execution)
2. Multi-turn conversations with persistent history
3. Tool selection logic and conditions
4. Fallback handling for failed tools
5. Cost optimization (track LLM calls)

### Manual vs Agent Component

**Manual (this notebook):**
- ✅ Full control over routing
- ✅ Understand every step
- ❌ More code to maintain

**Agent Component:**
- ✅ Less boilerplate
- ✅ Built-in error handling
- ❌ Less control

Choose based on your needs!